In [1]:
import argparse
import json
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor, hooks
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, print_csv_format

import os
import json
import yaml
import torch
import random
import detectron2
import pandas as pd

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from train import get_dataset_dicts, get_dataset_dicts_with_regions, random_visu, setup_cfg, MyTrainer, load_json_arr, find_best_model

import shutil

In [2]:
### FOLDER PATH ###

dataset_eval = "test"
dataset_eval_output = "test_output"
annotation_json = "via_region_data.json"
results_file = "results.json"
experiment = "../NSO/output/eval_full_augs_dc"
conf_path = "../NSO/output/eval_full_augs_dc/NSOD2cfg_1000_169r_1000pix_noBT_001LR_4BS_128BSPI_eval.yaml"
model_path = experiment + "/model_2.pth"
filter_empty_annot = False


annotation_output = os.path.splitext(annotation_json)[0]
out_dir = os.path.join(experiment, dataset_eval_output, annotation_output)

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    print ("The test directory is created!")

The test directory is created!


In [3]:
### Indicate to D2 how to read dataset if not in coco format: ###

# D2 metadata: name of classes and colors of annotations
classes = ["asset"]
colors = [(249, 180, 45)]

# Register dataset and metadata
for d in ["train", "val", "test"]:
    DatasetCatalog.register(d, lambda d=d:
                            get_dataset_dicts(os.path.join("../NSO", d), annotation_json))
    # Key-value mapping to interpret what’s in the dataset: names of classes, colors of classes
    MetadataCatalog.get(d).thing_classes = classes
    MetadataCatalog.get(d).thing_colors = colors

In [4]:
cfg = get_cfg()
# keep those 2 to avoid errors of MIN_SIZE_TRAIN = 800 and resize not found
#change in config: 
#MIN_SIZE_TRAIN: !!python/tuple
#- 1000
cfg.INPUT.MIN_SIZE_TRAIN = (1000,)
cfg.RESIZE= False
cfg.merge_from_file(conf_path)
cfg.INPUT.MIN_SIZE_TRAIN = (1000,)

cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.01
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = filter_empty_annot

predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator(dataset_eval, output_dir=out_dir)

val_loader = build_detection_test_loader(cfg, dataset_eval)
results = inference_on_dataset(predictor.model, val_loader, evaluator)

with open(os.path.join(out_dir, results_file), 'w') as f:
    json.dump(results, f)
print("Results dumped")

Loading config ../NSO/output/eval_full_augs_dc/NSOD2cfg_1000_169r_1000pix_noBT_001LR_4BS_128BSPI_eval.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
/scistor/ivm/jpl204/miniconda3/envs/nso_fix/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/scistor/ivm/jpl204/miniconda3/envs/nso_fix/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/Tenso

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.234
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.144
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.132
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.182
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.187
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.031
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.161
 Average Recall     (AR) @[ IoU=0.50:0.

In [5]:
#out_dir = '../NSO/output/lr001_BS4_empty-annot-50/eval'
# Load the JSON pred file into a DataFrame
df_pred = pd.read_json(os.path.join(out_dir, 'coco_instances_results.json'))
df_pred = df_pred[df_pred['score'] > 0.3] 

# Load the JSON ground truth file into a DataFrame

# CHANGE VAL OR TEST
# Extract the "images" key and create a DataFrame from it

# Ground thruth OG
df_truth_og = pd.read_json(os.path.join(os.path.join("../NSO", dataset_eval), annotation_json))
with open(os.path.join(out_dir, 'test_coco_format.json'), 'r') as j:
     data = json.loads(j.read())
images_data = data.get('images', [])  # Get the list of "images" objects
annotation_data = data.get('annotations', [])  # Get the list of "categories" objects
# Create a DataFrame from the valid "images" objects
df_images = pd.DataFrame(images_data)
df_annotations = pd.DataFrame(annotation_data)
#Drop duplicates: where are they comming from???? =>val_coco is automatically done by D2 check if where nitially in dataset
df_annotations = df_annotations.drop_duplicates(subset=['bbox'])

In [6]:
df_truth = pd.merge(df_images, df_annotations, left_on='id', right_on='image_id', how='left')



In [7]:
# #drop duplicates => why are they there?
#df_truth = df_truth.drop_duplicates(subset=['bbox'])
df_eval = pd.merge(df_truth, df_pred, left_on='id_x', right_on='image_id', how='left')
# df_eval = df_eval.drop_duplicates(subset=['segmentation_x'])
# #draop images with several annotation, just need to keep one
df_eval = df_eval.drop_duplicates(subset=['id_x'])
df_eval['truth'] = None
df_eval['pred'] = None

In [8]:
df_eval['truth'] = np.where(df_eval.bbox_x.isnull(), 0, 1)
df_eval['pred'] = np.where(df_eval.score.isnull(), 0, 1)

In [9]:
from sklearn.metrics import confusion_matrix

#ravel to flatten conf matrix to 1D insted of 2D
tn, fp, fn, tp = confusion_matrix(df_eval['truth'], df_eval['pred']).ravel()
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2 * (precision_score * recall_score) / (precision_score + recall_score)
print ("Fscore: ", f1_score)

Fscore:  0.7019867549668874


In [10]:
# Convert int64 values to native Python int
tn = int(tn)
fp = int(fp)
fn = int(fn)
tp = int(tp)

metrics_dict = {
    'tn': tn,
    'fp': fp,
    'fn': fn,
    'tp': tp,
    'precision_score': precision_score,
    'recall_score': recall_score,
    'f1_score': f1_score
}

# Define the file path
json_metrics = 'metrics_tiles.json'
json_path = os.path.join(out_dir, json_metrics)

# Save the metrics to a JSON file
with open(json_path, 'w') as json_file:
    json.dump(metrics_dict, json_file, indent=4)

print(f'Metrics saved to {json_file}')
print(f1_score)

Metrics saved to <_io.TextIOWrapper name='../NSO/output/eval_full_augs_dc/test_output/via_region_data/metrics_tiles.json' mode='w' encoding='UTF-8'>
0.7019867549668874
